# Are you getting spam messages in your inbox???
<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTLLx3jilwIExEUyOBnj_pRKGS0E-IEel_jTVE8eSzVgBsYeH1Q" width="500px" align="left">

**What are Spam messages ? **

Spam messages are unsolicited, usually commercial messages (such as e-mails, text messages, or Internet postings) sent to a large number of recipients or posted in a large number of places.

**Problem Statement : ** Accurately classifing spam messages by building a predictive model. 

**Solution : ** By doing data preprocessing and predictive analysis using naive bayes, we can achieve the goal of accurately classifing spam messages.

**Agenda :**
1. [Import libraries](#1)
2. [Load the data](#2)
3. [Data Cleaning and Data Preprocessing](#3)
    * [Dropping null data](#4)
    * [Renaming columns](#5)    
    * [Counting values in `label`](#6)
    * [Checking shape of the dataset](#7)
    * [Dropping duplicates](#8)
    * [Checking shape of the dataset after dropping duplicates](#9)
    * [Mapping `label`](#10)
    * [Dropping `label` column](#11)
    * [Removing Punctuations](#12)
    * [Removing Stopwords](#12)
4. [Data Visualization](#13)
    * [WordCloud for spam messages](#14)
    * [WordCloud for ham messages](#15)
5. [Feature Extraction](#16)
6. [Train and Test split](#17)
7. [Predictive Analysis](#18)
8. [Model Evaluation](#19)
9. [Test on random data](#20)
10. [Conclusion](#21)

Let's start here!

## 1. Import libraries<a id="1"></a>

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import string
import pandas as pd
%matplotlib inline

## 2. Load the data<a id="2"></a>

In [ ]:
# loading data

mails = pd.read_csv("../input/sms-spam-collection-dataset/spam.csv",encoding = 'latin-1')
mails.head()

## 3. Data Cleaning and Data preprocessing<a id="3"></a>
Data cleaning - In this step, we'll drop columns, rename columns, map columns or drop duplicate values.

Preprocessing - Preprocessing is one of the major steps when we are dealing with any kind of text models. During this stage we have to look at the distribution of our data, what techniques are needed and how deep we should clean.

### Dropping null data <a id="4"></a>

In [ ]:
# drop the null columns namely Unnamed: 2, Unnamed: 3 and Unnamed: 4

mails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1,inplace=True)
mails.head()

### Renaming columns <a id="5"></a>

In [ ]:
# renaming the columns v1 and v2 as labels and message

mails.rename(columns = {'v1':'labels', 'v2':'message'}, inplace=True)
mails.head()

### Counting values in `label`<a id="6"></a>

In [ ]:
# count of labels

mails['labels'].value_counts()

### Checking shape of the data<a id="7"></a>

In [ ]:
mails.shape

### Dropping duplicates<a id="8"></a>

In [ ]:
# now, we will see if our dataset contains duplicates, we will drop the duplicates

mails.drop_duplicates(inplace=True)

### Checking for the shape of the data after dropping duplicates<a id="9"></a>

In [ ]:
# after droping duplicates let's see the shape of dataset again

mails.shape

As we see our dataset had contained 403 duplicates.

In [ ]:
# check for any null values in the dataset

mails.isnull().sum()

Thus, there is no null value in the data.

### Mapping `label`<a id="10"></a>

In [ ]:
# mapping the labels as 0 or 1
# 0 for ham and 1 for spam

mails['label'] = mails['labels'].map({'ham': 0, 'spam': 1})
mails.head()

### Dropping `label` column<a id="11"></a>

In [ ]:
# now, labels column is of no use so we will drop the labels columns

mails.drop(['labels'], axis=1, inplace=True)
mails.head()

### Tokenzation<a id='12'></a>
We will tokenize our dataset using some techniques. Tokenization basically refers to splitting up a larger body of text into smaller lines, words or even creating words.

Punctuations are the unnecessary symbols that are in our corpus documents.

Stop words are the most commonly occurring words which don’t give any additional value to the document vector. in-fact removing these will increase computation and space efficiency.

In [ ]:
def process_text(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
#show the tokenization 

mails['message'].head().apply(process_text)

## 4. Data Visualization<a id = "13"></a>

Now, we will see frequently used words using WordCloud.

### WordCloud for spam messages<a id='14'></a>

In [ ]:
spam_words = ' '.join(list(mails[mails['label'] == 1]['message']))
spam_wc = WordCloud(width = 512,height = 512).generate(spam_words)
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(spam_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

### WordCloud for ham messages<a id='15'></a>

In [ ]:
ham_words = ' '.join(list(mails[mails['label'] == 0]['message']))
ham_wc = WordCloud(width = 512,height = 512).generate(ham_words)
plt.figure(figsize = (10, 8), facecolor = 'k')
plt.imshow(ham_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

## 5. Feature Extraction<a id = "16"></a>
The CountVectorizer provides a simple way to both tokenize a collection of text documents and build a vocabulary of known words, but also to encode new documents using that vocabulary.

In [ ]:
# Convert a collection of text documents to a matrix of token counts
# message_bow stands for bag of words

from sklearn.feature_extraction.text import CountVectorizer
x = mails['message']
y = mails['label']
cv = CountVectorizer()
x= cv.fit_transform(x)

## 6. Train and Test split<a id = "17"></a>

In [ ]:
# split the data into 80% training and 20% testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

In [ ]:
#shape of data after Vectorization

x.shape

## 7. Predictive analysis<a id = "18"></a>
**Q.** Why we are using Naive Bayes?

Naive Bayes is commonly applied to text classification. Naive Bayes classifiers work by correlating the use of tokens with spam and non-spam e-mails and then using Bayes' theorem to calculate a probability that an email is or is not spam.

The “Naive” assumption that the Naive Bayes classifier makes is that the probability of observing a word is independent of each other. The result is that the “likelihood” is the product of the individual probabilities of seeing each word in the set of Spam or Ham emails.

In [ ]:
# Create and train the naive Bayes classifier
# The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification)

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(X_train, y_train)

## 8. Model Evaluation<a id = "19"></a>

In [ ]:
# Evaluate the model on the test data set

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(X_test)
print(classification_report(y_test, pred))
print()
print('Confusion Matrix:\n',confusion_matrix(y_test, pred))
print()
print('Accuracy : ',accuracy_score(y_test, pred))

In [ ]:
# print the predictions
print(classifier.predict(X_test))

# print the actual values
print(y_test.values)

## 9. Test on random data<a id='20'></a>

In [ ]:
def sms(text):
    
    # creating a list of labels
    lab = ['not spam','spam'] 
    
    # perform tokenization
    x = cv.transform(text).toarray()
    
    # predict the text
    p = classifier.predict(x)
    
    # convert the words in string with the help of list
    s = [str(i) for i in p]
    a = int("".join(s))
    
    # show out the final result
    res = str("This message is looking: "+ lab[a])
    print(res)

In [ ]:
sms(['Congratulations, your entry into our contest last month made you a WINNER! goto our website to claim your price! You have 24 hours to claim.'])

In [ ]:
sms(['Your mobile number has won 1,615,000 million pounds in Apple iPhone UK. For claim email your name, country, occupation.'])

In [ ]:
sms(['Our Summer Sale is live from 15th to 17th june! Get 40% off on select products. visit the store now.'])

In [ ]:
sms(['Hey there! I am using kaggle'])

In [ ]:
sms(['Did you here about the new tv show?'])

In [ ]:
sms(['I am free after 11:00 in morning, meet you soon!'])

## 10. Conclusion<a id='21'></a>
We’ve come a long way in your journey to learn about Spam Classification using NLTK(natural language toolkit). Here we are using the Naive Bayes Classification and extracting the word using word-count algorithm. After calculation we find that  Naive Bayes' Classification has more accuracy support. The error rate is very low when we are using the Naive Bayes Classification. Thus, using naive bayes algorithm we can classify ham and spam messages with the accuracy of 97%.